In [10]:
%pip install pandas sentence-transformers langchain_community


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
   -------------------------------- ------- 184.3/227.1 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 227.1/227.1 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 16.3 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/11.0 MB 22.4 MB/s eta 0:00:01
   ------- -------------------------------- 2.2/11.0 MB 15.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/11.0 MB 18.2 MB/s eta 0:00:01
   -------------- -----------


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from langchain_community.llms import Ollama

# Inicializa el modelo de Ollama
llm = Ollama(
    model="llama3:latest",
    system="Tienes que completar las oraciones que pasa el usuario de una forma que tenga sentido y sea concisa."
)

# Lee el archivo CSV y extrae el contenido de interés
csv_file = 'generated_responses.csv'  # Reemplaza con la ruta a tu archivo CSV
df = pd.read_csv(csv_file, delimiter='|')

# Supongamos que la columna 'text' contiene los textos de interés
texts = df['text'].tolist()

# Función para ajustar la entrada al modelo basado en los textos del CSV
def generate_prompt(user_input, examples):
    prompt = f"Ingresa el texto: {user_input}\n\n"
    prompt += "Basado en los siguientes ejemplos, completa la oración de manera similar:\n\n"
    for example in examples:
        prompt += f"- {example}\n"
    return prompt

In [4]:
# Obtiene el texto del usuario
user_text = input("Ingresa el texto: ")

# Genera el prompt con ejemplos
prompt = generate_prompt(user_text, texts[10:])  # Usamos los ultimos 10 ejemplos, puedes ajustar según sea necesario

# Envía la consulta y procesa la respuesta en tiempo real
response = ""

print(end=' ', flush=True)

for chunk in llm.stream(prompt):
    for char in chunk:
        print(char, end='', flush=True)
        response += char

# Imprime una nueva línea al final para evitar sobrescribir en la consola
print()

In [1]:
import pandas as pd
from langchain_community.llms import Ollama

# Inicializa el modelo de Ollama
llm = Ollama(
    model="llama3:latest",
    system="Tienes que completar las oraciones que pasa el usuario de una forma que tenga sentido y sea concisa."
)

# Lee el archivo CSV y extrae el contenido de interés
csv_file = 'generated_responses.csv'  # Reemplaza con la ruta a tu archivo CSV
df = pd.read_csv(csv_file, delimiter='|')

# Supongamos que la columna 'text' contiene los textos de interés
texts = df['text'].tolist()

# Función para ajustar la entrada al modelo basado en los textos del CSV
def generate_prompt(user_input, examples):
    prompt = f"Ingresa el texto: {user_input}\n\n"
    prompt += "Basado en los siguientes ejemplos, completa la oración de manera similar:\n\n"
    for example in examples:
        prompt += f"- {example}\n"
    return prompt

# Obtiene el texto del usuario
user_text = input("Ingresa el texto: ")

# Genera el prompt con ejemplos (usamos menos ejemplos para reducir el tiempo de respuesta)
prompt = generate_prompt(user_text, texts[-5:])  # Usamos los últimos 5 ejemplos

# Envía la consulta y procesa la respuesta en tiempo real
response = ""

print(end=' ', flush=True)

for chunk in llm.stream(prompt):
    for char in chunk:
        print(char, end='', flush=True)
        response += char

# Imprime una nueva línea al final para evitar sobrescribir en la consola
print()

 Mi nombre es Sofía y estoy aquí para compartir mis experiencias y aprendizajes con ustedes.
